In [1]:
import os
import sys
root_folders = ['..', '../..', './']
root_folders = [os.path.abspath(os.path.join(x)) for x in root_folders]
for path in root_folders:
    if path not in sys.path:
        sys.path.append(path)
import nltk
%load_ext autoreload
%autoreload 2

from deeplavrov.deeplavrov.vocabulary.vocabulary import WordVocabEncoder

In [2]:
with open('1mcorpus/corpus.en_ru.1m.ru', encoding='utf-8') as f:
    ru = f.read()

In [3]:
import collections

In [4]:
c = collections.Counter(ru.split(' '))

In [5]:
len(c)

1924571

Вот сколько токенов всего!!!

In [ ]:
# from razdel import tokenize
# tokens = tokenize("Обычно я просыпался в 7:30, спускался вниз и видел что дверь в дом открыта, на кухне и в гостиной стоит 600 банок пива и дома никого нет.")
# print([t.text for t in tokens])

In [6]:
encoder = WordVocabEncoder(max_seq_len=117, word_min_freq=1)

In [7]:
source = '1mcorpus/corpus.en_ru.1m.en'
target = '1mcorpus/corpus.en_ru.1m.ru'

In [8]:
encoder.build(source, target)

Building vocab from 1mcorpus/corpus.en_ru.1m.en took 2.1585755348205566 second(s)
Building vocab from 1mcorpus/corpus.en_ru.1m.ru took 2.3104419708251953 second(s)


In [9]:
ru_vocab = encoder.target_vocab

In [10]:
len(ru_vocab)

46454

Как так!?

In [11]:
'Багрий' in ru_vocab

False

In [12]:
'Багрий' in c

True

Как так (